# Scrape Geschäfte

In [59]:
import urllib
import calendar
import requests
from pathlib import Path
import glob, os
import xml.etree.ElementTree as ET
import copy
import pandas as pd
import json

## 1. Load all Geschäfte

In [2]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh5/cdws/Index/GESCHAEFT/searchdetails?q=%s&l=de-CH'

year_from = 2001
year_to = 2022
print("Years: %s - %s" % (year_from, year_to))
for year in range(year_from, year_to):
    for i in range(0, 3):
        #print("%s %s" % (year, i))

        # First day
        d_start = '%s-%02d-01 00:00:00' % (year, i * 4 + 1)

        # Last day
        l = calendar.monthrange(year, (i + 1) * 4)[-1]
        d_end = '%s-%02d-%s 23:59:00' % (year, (i + 1) * 4, l)

        q = '((beginn_start >= "%s" and beginn_end <= "%s")) sortBy beginn_start/sort.ascending krnr/sort.ascending' % (d_start, d_end)

        r = requests.get(url % urllib.parse.quote(q))
        with open(Path('../export/GESCHAEFT/%s_%s.xml' % (year, i)), 'wb') as f:
            f.write(r.content)

Years: 2001 - 2022


## 2. Create DataFrame from XML

In [57]:
# Parse all XML and create Dataframe
ns = {
    'g': 'http://www.cmiag.ch/cdws/Geschaeft'
    }

records = []

# Open all xml-files
for f in glob.glob(str(Path('../export/GESCHAEFT/*.xml'))):
    tree = ET.parse(f)
    root = tree.getroot()

    try:

        for g in root.findall('.//g:Geschaeft', ns):

            if g.find('g:PendentBeiNeu/g:Organisationseinheit/g:Name', ns) == None:
                continue

            # Find Erstunterzeichner:in
            erstunterzeichner = {'name': '', 'vorname': ''}
            for beteiligung in g.findall('g:Beteiligungen/g:Beteiligung', ns):
                if (beteiligung.find('g:Rolle', ns).text != None) and ('erstunterzeichner' in beteiligung.find('g:Rolle', ns).text.lower()):
                    erstunterzeichner['vorname'] = beteiligung.find('g:Vorname', ns).text
                    erstunterzeichner['name'] = beteiligung.find('g:Name', ns).text
                    erstunterzeichner['istkantonsrat'] = beteiligung.find('g:IstKantonsrat', ns).text
                    erstunterzeichner['partei'] = beteiligung.find('g:ParteiKurzname', ns).text
                    break

            if erstunterzeichner['name'] == '':
                continue

            records.append({
                'KRNr': g.find('g:KRNr', ns).text,
                'VorlageNr': g.find('g:VorlageNr', ns).text,
                'Geschaeftsart': g.find('g:Geschaeftsart', ns).text,
                'BehandelndeKommission': g.find('g:BehandelndeKommission', ns).text,
                'BehandelndeKommissionKurzname': g.find('g:BehandelndeKommissionKurzname', ns).text,
                'Direktion': g.find('g:Direktion', ns).text,
                'DirektionKurzname': g.find('g:DirektionKurzname', ns).text,
                'Start': g.find('g:Beginn/g:Start', ns).text,
                'End': g.find('g:Beginn/g:End', ns).text,
                'Zusammenfassung': g.find('g:Zusammenfassung', ns).text,
                'status': g.find('g:PendentBeiNeu/g:Organisationseinheit/g:Name', ns).text,
                'erstunterzeichnerVorname': erstunterzeichner['vorname'],
                'erstunterzeichnerName': erstunterzeichner['name'],
                'erstunterzeichnerIstKantonsrat': erstunterzeichner['istkantonsrat'],
                'erstunterzeichnerPartei': erstunterzeichner['partei'],
            })
            
    except:
        print(f)
        print(g.find('g:KRNr', ns).text)
        raise

df_raw = pd.DataFrame(records)
df_raw = df_raw.sort_values('Start')

## Join with Member Data

In [60]:
with open(Path('../export/mitglieder.json'), encoding='utf-8') as f:
    mitglieder = json.load(f)

df_mitglieder = pd.DataFrame(list(map(lambda m: {'name': m['name'], 'vorname': m['vorname'], 'geschlecht': m['geschlecht'], 'jahrgang': m['jahrgang']}, mitglieder)))

In [76]:
df = df_raw.merge(df_mitglieder, left_on = ['erstunterzeichnerVorname', 'erstunterzeichnerName'], right_on = ['vorname', 'name'])
df.drop(columns=['name', 'vorname'], inplace=True)

df.to_csv(Path('../export/geschaefte.csv'), index=False)